In [1]:
import pandas as pd


In [9]:
df=pd.read_csv('../_data/operations_inscope.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58815 entries, 0 to 58814
Data columns (total 58 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   op_id              58815 non-null  int64  
 1   subject_id         58815 non-null  int64  
 2   hadm_id            58815 non-null  int64  
 3   opdate             58815 non-null  int64  
 4   age                58815 non-null  int64  
 5   sex                58815 non-null  object 
 6   weight             58180 non-null  float64
 7   height             58461 non-null  float64
 8   race               58815 non-null  object 
 9   asa                56842 non-null  float64
 10  emop               58815 non-null  int64  
 11  department         58815 non-null  object 
 12  antype             58815 non-null  object 
 13  icd10_pcs          58815 non-null  object 
 14  category_desc      58815 non-null  object 
 15  desc_short         58815 non-null  object 
 16  category_id        588

## Objective: Identify LOS Outliers withing each surgery type

We use `groupby`('icd10_pcs') groups the data by the unique values in the 'icd10_pcs'.  
 For each group, `transform` applies the calculate_percentile_75 function to values within that group.   
 This group-specific 75th percentile is then used to determine if a value is an outlier within its category.

Lastly, the is_outlier column is created to indicate with 1 if the record is an outlier in its category or 0 otherwise. We also clean up by dropping the intermediate 'percentile_75' column.


In [20]:
import pandas as pd
import numpy as np

# Assuming df is your DataFrame, 'column_name' is the column you're examining,
# and 'category_column' is the column with the categorical data
df['LOS']=(df['discharge_time']-df['orout_time'])/1440 # LOS is time following surgery to dishcarge converted to days. 

# Define a function to calculate the 75th percentile for a series
def calculate_percentile_75(series):
    return series.quantile(0.75)

# Apply the function to each group's 'icd10_pcs' and create a new column for the 75th percentile
df['percentile_75'] = df.groupby('icd10_pcs')['LOS'].transform(calculate_percentile_75)

# Create a new column with 0/1 based on the condition, using the group-specific 75th percentile
df['is_outlier'] = np.where(df['LOS'] > df['percentile_75'], 1, 0)

# Drop the 'percentile_75' column if it's no longer needed
df.drop('percentile_75', axis=1, inplace=True)

df
# Optionally, if you still need the filtered DataFrame without outliers, you can filter the DataFrame
df_no_outlier = df[df['is_outlier'] == 0]


,op_id,subject_id,hadm_id,opdate,age,weight,height,asa,emop,orin_time,...,hb,hco3,lymphocyte,platelet,potassium,sodium,total_bilirubin,wbc,LOS,is_outlier
count,4.422700e+04,4.422700e+04,4.422700e+04,4.422700e+04,44227.000000,43778.000000,44016.000000,42955.000000,44227.000000,4.422700e+04,...,12754.000000,9944.000000,10416.000000,11972.000000,17264.000000,17135.000000,8134.000000,11744.000000,44227.000000,44227.0
mean,4.499775e+08,1.501176e+08,2.501736e+08,2.041050e+05,56.058629,62.245374,161.595352,1.727273,0.097316,2.048240e+05,...,12.093747,23.863936,21.713297,212.305212,3.924631,139.196177,0.957155,8.834136,3.924024,0.0
std,2.886688e+07,2.885716e+07,2.890335e+07,6.263424e+05,15.463223,11.911047,83.332957,0.627437,0.296391,6.263466e+05,...,1.958106,2.925450,12.177941,72.319081,0.452148,3.616322,1.083636,4.076440,4.742772,0.0
min,4.000005e+08,1.000008e+08,2.000024e+08,0.000000e+00,20.000000,-1.000000,-1.000000,1.000000,0.000000,1.000000e+01,...,4.100000,4.600000,0.000000,19.000000,2.000000,115.000000,0.100000,0.560000,-1.385417,0.0
25%,4.250528e+08,1.250958e+08,2.251778e+08,1.440000e+03,45.000000,54.000000,155.000000,1.000000,0.000000,2.030000e+03,...,10.700000,22.100000,11.600000,167.000000,3.600000,137.000000,0.500000,5.930000,1.302083,0.0
50%,4.499423e+08,1.503455e+08,2.503584e+08,2.880000e+03,60.000000,61.000000,161.000000,2.000000,0.000000,3.345000e+03,...,12.100000,23.900000,20.500000,207.000000,3.900000,140.000000,0.700000,7.940000,2.482639,0.0
75%,4.750742e+08,1.750075e+08,2.751091e+08,1.008000e+04,70.000000,69.000000,167.000000,2.000000,0.000000,1.063500e+04,...,13.400000,25.700000,30.400000,250.000000,4.200000,141.000000,1.000000,10.892500,5.284722,0.0
max,4.999976e+08,1.999994e+08,2.999989e+08,5.132160e+06,90.000000,354.000000,17409.000000,6.000000,1.000000,5.133090e+06,...,25.500000,46.800000,97.000000,1113.000000,7.950000,181.000000,24.800000,85.620000,79.399306,0.0


In [17]:
import plotly.express as px


fig = px.violin(df_no_outlier, y="LOS", x="icd10_pcs", color="sex", box=True,hover_data=df.columns)
fig.show()

In [23]:
## Create flag if subject went to ICU

df['icu_visit'] = df['icuin_time'] > 0
df[df['icu_visit']== True]

## 7250 subjects went to ICU post op. 



,op_id,subject_id,hadm_id,opdate,age,sex,weight,height,race,asa,...,hco3,lymphocyte,platelet,potassium,sodium,total_bilirubin,wbc,LOS,is_outlier,icu_visit
0,446270725,158995752,257857903,0,70,M,43.0,169.0,Asian,NaN,...,18.0,38.1,124.0,4.2,140.0,0.5,NaN,47.920139,1,True
5,487807079,145247513,259299532,74880,25,M,47.0,172.0,Asian,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.131944,1,True
13,480242953,187733661,238035661,10080,55,F,46.0,152.0,Asian,2.0,...,27.9,NaN,NaN,3.3,139.0,NaN,NaN,16.302083,0,True
17,432945014,126077562,272790506,2880,50,F,61.0,149.0,Asian,2.0,...,NaN,NaN,NaN,NaN,NaN,3.8,NaN,17.305556,0,True
18,461524976,194549663,293384214,2880,50,M,75.0,171.0,Asian,2.0,...,18.8,NaN,NaN,4.3,132.0,NaN,NaN,11.069444,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58794,475125418,128375644,289406001,24480,50,F,61.0,155.0,Asian,3.0,...,24.2,NaN,NaN,4.3,135.0,NaN,NaN,18.312500,0,True
58795,486837329,104340242,298141008,105120,65,M,48.0,161.0,Asian,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106.173611,1,True
58802,447857589,163754043,274747413,11520,70,M,59.0,168.0,Asian,4.0,...,24.3,NaN,NaN,5.2,135.0,NaN,NaN,15.361111,1,True
58803,494319208,150497300,289152391,3483360,80,F,52.0,150.0,Asian,4.0,...,NaN,10.6,227.0,3.3,141.0,0.5,15.79,60.829861,1,True
